In [4]:
import pandas as pd
import time
import datetime
import csv
import numpy as np
pd.set_option('mode.chained_assignment',  None)

In [5]:
# Read Combined Dataset downloaded from the github
epl = pd.read_csv("/Users/bradyhong/Downloads/SoccerPredictions-afe206f8403d87d12fa83bcb62d708820034eac7/Data/Combined_Data.csv")

In [6]:
# Get the all the possible teams
epl_team = set()
epl.home_team
for i in epl.home_team:
    epl_team.add(i)

In [9]:
# Making features and aggregate them to the combined dataset
def recency(depth):
    for a in range(depth):
        # Making features
        epl["t-"+str(a+1)+" home_attacking"] = np.nan
        epl["t-"+str(a+1)+" home_defending"] = np.nan
        epl["t-"+str(a+1)+" home_home"] = np.nan
        epl["t-"+str(a+1)+" home_strength"] = np.nan
        epl["t-"+str(a+1)+" home_posession"] = np.nan
        epl["t-"+str(a+1)+" home_shots_on_target"] = np.nan
        epl["t-"+str(a+1)+" home_clearance"] = np.nan
        epl["t-"+str(a+1)+" home_tackles"] = np.nan
        epl["home_strength"] = np.nan
        epl["t-"+str(a+1)+" away_home"] = np.nan
        epl["t-"+str(a+1)+" away_attacking"] = np.nan
        epl["t-"+str(a+1)+" away_defending"] = np.nan
        epl["t-"+str(a+1)+" away_strength"] = np.nan
        epl["t-"+str(a+1)+" away_posession"] = np.nan
        epl["t-"+str(a+1)+" away_shots_on_target"] = np.nan
        epl["t-"+str(a+1)+" away_clearance"] = np.nan
        epl["t-"+str(a+1)+" away_tackles"] = np.nan
        epl["away_strength"] = np.nan
    for i in epl_team:
        home = pd.DataFrame(epl[epl.home_team == i])
        away = pd.DataFrame(epl[epl.away_team == i])
        new_data = pd.concat([home, away]).sort_index() # Dataset for each team i
        index = new_data.index 
        for j in range(len(index)-depth):
            if new_data.loc[index[j+depth], "home_team"] == i:
                count = 0
                for c in range(depth):
                    past = new_data.loc[index[j+depth-c-1]]
                    if i == past.home_team:
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_attacking"] = past.home_score
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_defending"] = past.away_score
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_home"] = 1
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_strength"] = past.home_strength     
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_posession"] = past.home_possession_percent
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_shots_on_target"] = past.home_shots_on_target
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_clearance"] = past.home_clearances
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_tackles"] = past.home_tackles
                        count = count + int(past.home_score) - int(past.away_score)
                    else:
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_attacking"] = past.away_score
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_defending"] = past.home_score
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_home"] = -1
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_strength"] = past.away_strength
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_posession"] = past.away_possession_percent
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_shots_on_target"] = past.away_shots_on_target
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_clearance"] = past.away_clearances
                        new_data.loc[index[j+depth], "t-"+str(c+1)+" home_tackles"] = past.away_tackles
                        count = count + int(past.away_score) - int(past.home_score)
                new_data.loc[index[j+depth], "home_strength"] = count/depth
            else:
                count = 0
                for d in range(depth):
                    past = new_data.loc[index[j+depth-d-1]]
                    if i == past.home_team:
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_attacking"] = past.home_score
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_defending"] = past.away_score
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_home"] = 1
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_strength"] = past.home_strength
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_posession"] = past.home_possession_percent
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_shots_on_target"] = past.home_shots_on_target
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_clearance"] = past.home_clearances
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_tackles"] = past.home_tackles
                        count = count + int(past.home_score) - int(past.away_score)
                    else:
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_attacking"] = past.away_score
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_defending"] = past.home_score
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_home"] = -1
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_strength"] = past.away_strength
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_posession"] = past.away_possession_percent
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_shots_on_target"] = past.away_shots_on_target
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_clearance"] = past.away_clearances
                        new_data.loc[index[j+depth], "t-"+str(d+1)+" away_tackles"] = past.away_tackles
                        count = count + int(past.away_score) - int(past.home_score)
                new_data.loc[index[j+depth], "away_strength"] = count/depth
        for x in index:
            epl.loc[x] = new_data.loc[x]

In [17]:
recency(depth = 9) # Run the recency feature with depth = 9

KeyboardInterrupt: 

In [ ]:
# aggregated file to csv file
epl.to_csv("/Users/bradyhong/Downloads/recency.csv")

In [12]:
# Read back from the csv file
recencys = pd.read_csv("/Users/bradyhong/Downloads/recency.csv")

In [15]:
# Fill the NAN with mean of each features
for i in range(len(recencys)):
    for j in range(9):
        recencys["t-"+str(j+1)+" home_attacking"].fillna(recencys["t-"+str(j+1)+" home_attacking"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" home_defending"].fillna(recencys["t-"+str(j+1)+" home_defending"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" home_home"].fillna(recencys["t-"+str(j+1)+" home_home"].mean(),inplace = True)    
        recencys["t-"+str(j+1)+" home_strength"].fillna(recencys["t-"+str(j+1)+" home_strength"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" away_attacking"].fillna(recencys["t-"+str(j+1)+" away_attacking"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" away_defending"].fillna(recencys["t-"+str(j+1)+" away_defending"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" away_home"].fillna(recencys["t-"+str(j+1)+" away_home"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" away_strength"].fillna(recencys["t-"+str(j+1)+" away_strength"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" home_posession"].fillna(recencys["t-"+str(j+1)+" home_posession"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" home_shots_on_target"].fillna(recencys["t-"+str(j+1)+" home_shots_on_target"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" home_clearance"].fillna(recencys["t-"+str(j+1)+" home_clearance"].mean(),inplace = True)    
        recencys["t-"+str(j+1)+" home_tackles"].fillna(recencys["t-"+str(j+1)+" home_tackles"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" away_posession"].fillna(recencys["t-"+str(j+1)+" away_posession"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" away_shots_on_target"].fillna(recencys["t-"+str(j+1)+" away_shots_on_target"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" away_clearance"].fillna(recencys["t-"+str(j+1)+" away_clearance"].mean(),inplace = True)
        recencys["t-"+str(j+1)+" away_tackles"].fillna(recencys["t-"+str(j+1)+" away_tackles"].mean(),inplace = True)
        

In [16]:
# filled null values version to csv file
recencys.to_csv("/Users/bradyhong/Downloads/recency_filled_null.csv")